In [38]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project, run_parallel_all_time


In [39]:
directory = directory_project
update_plate_info(directory)

In [7]:
update_plate_info(directory_scratch)

In [40]:
len(get_current_folders(directory))
all_folders = get_current_folders(directory)

In [29]:
plate = 22
plates = [22,10,76,26,152,102]

# folders = all_folders.loc[all_folders['Plate']==plate]
folders = all_folders.loc[all_folders['Plate'].isin(plates)]
print(len(folders))
# folders = all_folders.loc[all_folders['Plate']==plate*all_folders['/Analysis/skeleton_realigned_compressed.mat']]
folders = all_folders.loc[(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(all_folders['Plate'].isin(plates))]

print(len(folders))

351
336


In [41]:
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/nx_graph_pruned_width.p'])]
len(folders)


1748

In [65]:
folders = all_folders.loc[(all_folders['Plate']==785)&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]
folders = folders.loc[(folders['/Analysis/nx_graph_pruned_width.p']==False)]

In [5]:
folders = all_folders.loc[(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]
# folders = folders.loc[(folders['/Analysis/nx_graph_pruned_width.p']==False)]

In [28]:
folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
8,20201205_1320_Plate38,59,38,Carrot,A5,001P100N,Y,NaN,20201126,True,...,True,True,True,False,True,True,True,False,"05.12.2020, 13:20:",-
12,20201206_1327_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"06.12.2020, 13:27:",NaN
14,20210110_1144_Plate38,76,38,Carrot,A5,001P100N,Y,NaN,20210108,True,...,True,True,True,False,True,True,True,True,"10.01.2021, 11:44:",-
21,20210112_1120_Plate29,22,29,Carrot,A5,001P100N,Y,NaN,20210107,True,...,True,True,True,False,True,True,True,True,"12.01.2021, 11:20:",-
24,20201226_0742_Plate37,91,37,Carrot,A5,001P100N,Y,NaN,20201217,True,...,True,True,True,False,True,True,True,True,"26.12.2020, 07:42:",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,20211003_1129_Plate31,800,31,Carrot,A5,001P100N,Y,NaN,20210928,True,...,True,True,True,False,True,True,True,True,"03.10.2021, 11:29:",-
2688,20210916_2307_Plate17,88,17,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"16.09.2021, 23:07:",-
2689,20210924_1723_Plate22,787,22,Carrot,A5,001P100N,Y,NaN,20210922,True,...,True,True,True,False,True,True,True,True,"24.09.2021, 17:23:",-
2690,20211008_0616_Plate38,785,38,Carrot,A5,001P100N,Y,NaN,20210928,True,...,True,True,True,False,True,True,True,False,"08.10.2021, 06:16:",-


In [51]:
corrupted_rotation

1047    20201209_0926_Plate40
Name: folder, dtype: object

In [53]:
corrupted_rotation = folders.loc[folders['/Analysis/transform_corrupt.mat']]['folder']

In [28]:
corrupt_folder = np.min(folders.loc[folders['/Analysis/transform_corrupt.mat']]['folder'])
corrupt_folder

'20210927_2112_Plate22'

In [54]:
folder_list = list(folders['folder'])
folder_list.sort()
indexes = [folder_list.index(corrupt_folder) for corrupt_folder in corrupted_rotation]

In [55]:
limit = 100
indexes.sort()
indexes = [index for index in indexes if index<limit]
indexes += [limit]
indexes

[100]

In [29]:

folder_list.index(corrupt_folder)

23

In [36]:
folders = all_folders.loc[all_folders['folder']=='20211009_0617_Plate38']
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
1816,20211009_0617_Plate38,785,38,Carrot,A5,001P100N,Y,NaN,20210928,True,...,True,True,True,False,True,True,True,False,"09.10.2021, 06:17:",-


In [13]:
plates = set(folders['Plate'].values)
plate_without_node_id = [plate for plate in plates if not np.any(folders.loc[(folders['Plate']==plate)]['/Analysis/nx_graph_pruned_labeled.p'])]

In [14]:
plate_without_node_id

[]

In [8]:
folders = all_folders.loc[(all_folders['Plate'].isin(plate_without_node_id))&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]

In [23]:
plates_select = [69,59]
folders = all_folders.loc[(all_folders['Plate'].isin(plates_select))&(all_folders['/Analysis/nx_graph_pruned_width.p'])]

In [20]:
# folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(all_folders['/Analysis/nx_graph_pruned_labeled.p'])]
len(folders)


164

In [6]:
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(1-all_folders['/Analysis/nx_graph_pruned_labeled.p'])]
len(folders)

964

In [13]:
plates = list(set(list(folders['Plate'].values)))
len(plates)

19

In [12]:
plates = list(set(list(folders['Plate'].values)))
plates

[10,
 785,
 787,
 22,
 790,
 792,
 152,
 26,
 799,
 800,
 808,
 59,
 69,
 76,
 80,
 88,
 91,
 94,
 102]

In [75]:
plates = list(set(list(all_folders['Plate'].values)))
for plate in plates:
    print(plate,len(all_folders.loc[all_folders['Plate']==plate]))

nan 0
10 31
785 94
787 73
22 31
790 98
792 149
152 115
26 44
799 95
800 91
808 91
59 127
69 107
76 25
80 55
88 155
91 115
94 169
102 103
756 90


In [ ]:
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(all_folders['/Analysis/nx_graph_pruned_width.p'])]
len(folders)

In [11]:
dates = list(all_folders.loc[all_folders['Plate']==785]['folder'])
dates.sort()
dates

['20210929_1004_Plate38',
 '20210929_1204_Plate38',
 '20210929_1404_Plate38',
 '20210929_1604_Plate38',
 '20210929_1804_Plate38',
 '20210929_2004_Plate38',
 '20210929_2204_Plate38',
 '20210930_0004_Plate38',
 '20210930_0204_Plate38',
 '20210930_0404_Plate38',
 '20210930_0604_Plate38',
 '20210930_0804_Plate38',
 '20210930_1004_Plate38',
 '20210930_2029_Plate38',
 '20210930_2210_Plate38',
 '20211001_0210_Plate38',
 '20211001_0410_Plate38',
 '20211001_0610_Plate38',
 '20211001_0810_Plate38',
 '20211001_1010_Plate38',
 '20211001_1210_Plate38',
 '20211002_1543_Plate38',
 '20211002_1744_Plate38',
 '20211002_1944_Plate38',
 '20211002_2144_Plate38',
 '20211002_2344_Plate38',
 '20211003_0143_Plate38',
 '20211003_0344_Plate38',
 '20211003_0543_Plate38',
 '20211003_0744_Plate38',
 '20211003_0944_Plate38',
 '20211003_1144_Plate38',
 '20211003_1344_Plate38',
 '20211003_1544_Plate38',
 '20211003_1744_Plate38',
 '20211003_1944_Plate38',
 '20211003_2144_Plate38',
 '20211003_2344_Plate38',
 '20211004_0

***Create graphs***

In [72]:
num_parallel = 100
time = '1:00:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders, num_parallel, time,'extract_nx')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 59034
Submitted batch job 59035


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [69]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])&(all_folders['Plate'].isin(plates))]
len(folders)

336

***Extract Width***

In [35]:
num_parallel = 50 
time = '24:00:00'
skip = False
resolution = 50
args = [directory,skip,resolution]
run_parallel('extract_width.py',args,folders,num_parallel,time,'extract_width2',cpus=128)

Submitted batch job 61718
Submitted batch job 61719


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [41]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(1-all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))&(all_folders['/Analysis/nx_graph_pruned.p'])]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

0

In [4]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

336

***Identify Nodes***

In [42]:
num_parallel = 5
time = '48:00:00'
limit = 100
args=[directory,limit]
run_parallel_all_time('extract_nodes.py',args,folders,num_parallel,time,'node_id',cpus = 128)

Submitted batch job 61733
Submitted batch job 61734
Submitted batch job 61735
Submitted batch job 61736


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [6]:
folders.columns

Index(['folder', 'Plate', 'PrincePos', 'root', 'strain', 'medium', 'split',
       'Temp', 'CrossDate', '/Img/TileConfiguration.txt.registered',
       '/Analysis/skeleton_compressed.mat',
       '/Analysis/skeleton_masked_compressed.mat',
       '/Analysis/skeleton_pruned_compressed.mat', '/Analysis/transform.mat',
       '/Analysis/transform_corrupt.mat',
       '/Analysis/skeleton_realigned_compressed.mat',
       '/Analysis/nx_graph_pruned.p', '/Analysis/nx_graph_pruned_width.p',
       '/Analysis/nx_graph_pruned_labeled.p', 'date', 'Pbait'],
      dtype='object')

In [60]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_labeled.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

239

***Hyphae extraction***

In [19]:
num_parallel = 1
time = '4:00:00'
limit = 100
version = 8
args = [directory,limit,version]
run_parallel_all_time('hyphae_extraction.py',args,folders,num_parallel,time,'hyphae',cpus = 128)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 57496
Submitted batch job 57497
Submitted batch job 57498
Submitted batch job 57499
Submitted batch job 57500


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 57501
Submitted batch job 57502
Submitted batch job 57503
Submitted batch job 57504
Submitted batch job 57505
Submitted batch job 57506
Submitted batch job 57507
Submitted batch job 57508
Submitted batch job 57509
Submitted batch job 57510
Submitted batch job 57511


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [16]:
 0.2 * 210 * 10**14 *30 *1.8

2.268e+17